# Notebook Purpose

Setting up scripts to train CNN. This was saved as a python file and then used for launching training jobs.

In [1]:
#!/usr/bin/env python
from __future__ import print_function, division
# coding: utf-8

# In[1]:


import torch
import sys
import antialiased_cnns
# from torchvision import datasets, models, transforms

from PIL import Image
import torchvision

sys.path.append('/om5/user/smadan/training_scaffold_own/res/')
from models.models import get_model
from loader.loader import get_loader
import random
import pickle
import argparse
import wandb

/om5/user/smadan/training_scaffold_own/res/loader/multi_attribute_loader.py
/om5/user/smadan/training_scaffold_own/res/loader
/om5/user/smadan/training_scaffold_own/res/loader/loader.py
/om5/user/smadan/training_scaffold_own/res/loader


In [2]:
from torchvision import transforms

In [9]:
wandb_config = {}
wandb_config['num_epochs'] = 50
wandb_config['model_arch'] = 'TRULY_SHIFT_INVARIANT'
wandb_config['batch_size'] = 100
wandb_config['num_classes'] = 11
wandb_config['base_lr'] = 0.001
wandb_config['use_gpu'] = True
wandb_config['run_name'] = 'normalization_test'
wandb_config['pretrained'] = False
wandb_config['image_size'] = 224
wandb_config['dataset_name'] = 'imagenet'
wandb_config['normalize'] = True

wandb.login()

True

In [10]:
loader = get_loader('multi_attribute_loader_file_list_shapenet')

import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import copy
import os
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
import collections
from collections import OrderedDict
from tqdm.notebook import tqdm


input_img_size = wandb_config['image_size']

image_transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
])

loader_new = get_loader('multi_attribute_loader_file_list_imagenet')
file_list_root = '/om5/user/smadan/dataset_lists_openmind'
att_path = '/om5/user/xboix/data/ImageNet/imagenet_metadata.txt'
shuffles = {'train':False,'val':False,'test':False}

count=0

file_lists = {}
dsets = {}
dset_loaders = {}
dset_sizes = {}
for phase in ['train','val']:
    file_lists[phase] = "%s/%s_list_%s.txt"%(file_list_root,phase,wandb_config['dataset_name'])
    dsets[phase] = loader_new(file_lists[phase],att_path, image_transform)
    dset_loaders[phase] = torch.utils.data.DataLoader(dsets[phase], batch_size=wandb_config['batch_size'], shuffle = shuffles[phase], num_workers=2,drop_last=True)
    dset_sizes[phase] = len(dsets[phase])

In [11]:
def test_model(model, best_acc, best_model, configs):
    if configs.normalize == True:
        print('Images will be normalized')
    model.eval()

    running_corrects = 0
    iters = 0
    for data in tqdm(dset_loaders['test']):
        inputs, labels, image_paths = data
        if configs.use_gpu:
            inputs = inputs.float().cuda()
            labels = labels.long().cuda()
        else:
            print('WARNING: NOT USING GPU!')
            inputs = inputs.float()
            labels = labels.long()


        if configs.normalize == True:
            im_means = torch.mean(inputs.view(configs.batch_size, -1),dim=1).unsqueeze(1).unsqueeze(1).unsqueeze(1)
            im_stds = torch.std(inputs.view(configs.batch_size, -1),dim=1).unsqueeze(1).unsqueeze(1).unsqueeze(1)
            inputs = (inputs - im_means)/im_stds

        outputs = model(inputs)
        _, preds = torch.max(outputs.data, 1)

        iters += 1
        running_corrects += torch.sum(preds == labels.data)
        wandb.log({"train_running_corrects": running_corrects/float(iters*len(labels.data))})


    epoch_acc = float(running_corrects) / float(dset_sizes['test'])

    wandb.log({"test_accuracy": epoch_acc})

    if epoch_acc > best_acc:
        best_acc = epoch_acc
        best_model = copy.deepcopy(model)
    wandb.log({"best_accuracy": best_acc})

    save_path = '/om5/user/smadan/differentiable_graphics_ml/training_models/saved_models/%s.pt'%configs.run_name
    with open(save_path,'wb') as F:
        torch.save(best_model, F)

    return best_acc, best_model

In [12]:
def train_model(model, criterion, optimizer, configs):
    if configs.normalize == True:
        print('Images will be normalized')
    best_model = model
    best_acc = 0.0

    model.train()
    running_loss = 0.0
    running_corrects = 0
    iters = 0

    for data in tqdm(dset_loaders['train']):
        inputs, labels, image_paths = data
        if configs.use_gpu:
            inputs = inputs.float().cuda()
            labels = labels.long().cuda()
        else:
            print('WARNING: NOT USING GPU!')
            inputs = inputs.float()
            labels = labels.long()

        if configs.normalize == True:
            im_means = torch.mean(inputs.view(configs.batch_size, -1),dim=1).unsqueeze(1).unsqueeze(1).unsqueeze(1)
            im_stds = torch.std(inputs.view(configs.batch_size, -1),dim=1).unsqueeze(1).unsqueeze(1).unsqueeze(1)
            inputs = (inputs - im_means)/im_stds

        optimizer.zero_grad()
        outputs = model(inputs)
        _, preds = torch.max(outputs.data, 1)

        loss = criterion(outputs, labels)

        loss.backward()
        optimizer.step()

        iters += 1
        running_loss += loss.item()
        running_corrects += torch.sum(preds == labels.data)
        wandb.log({"train_running_loss": running_loss/float(iters*len(labels.data))})
        wandb.log({"train_running_corrects": running_corrects/float(iters*len(labels.data))})

    epoch_loss = float(running_loss) / dset_sizes['train']
    epoch_acc = float(running_corrects) / float(dset_sizes['train'])
    wandb.log({"train_accuracy": epoch_acc})
    wandb.log({"train_loss": epoch_loss})
    return model

In [13]:
def model_pipeline(model, criterion, optimizer, hyperparameters):
    with wandb.init(project="pytorch-test", config=hyperparameters):
        if hyperparameters['run_name']:
            wandb.run.name = hyperparameters['run_name']
        config = wandb.config
        best_model = model
        best_acc = 0.0

        print(config)

        print(config.num_epochs)
        for epoch_num in range(config.num_epochs):
            wandb.log({"Current Epoch": epoch_num})
            model = train_model(model, criterion, optimizer, config)
            best_acc, best_model = test_model(model, best_acc, best_model, config)

    return best_model

In [14]:
for data in dset_loaders['val']:
    inputs, labels, paths = data
    break

In [36]:
paths[26]

IndexError: tuple index out of range

In [ ]:
model = get_model(wandb_config['model_arch'],wandb_config['num_classes'])
model.cuda();

In [10]:
model.fc

Linear(in_features=512, out_features=11, bias=True)

In [11]:
child_count = 0
for child in model.children():
    if child_count < 9:        
        for param in child.parameters():
            param.requires_grad = False
    child_count += 1

In [44]:
for child in model.children():
    for param in child.parameters():
        print(param.requires_grad)

False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
True
True


In [40]:
optimizer = optim.Adam(model.parameters(), lr = wandb_config['base_lr'])

In [41]:
criterion = nn.CrossEntropyLoss()

In [42]:
# import math

In [43]:
# leftover_bad_files = []

In [45]:
running_loss = 0
num_iters = 0
for data in tqdm(dset_loaders['train']):
    inputs, labels, image_paths = data
    inputs = inputs.float().cuda()
    im_means = torch.mean(inputs.view(wandb_config['batch_size'], -1),dim=1).unsqueeze(1).unsqueeze(1).unsqueeze(1)
    im_stds = torch.std(inputs.view(wandb_config['batch_size'], -1),dim=1).unsqueeze(1).unsqueeze(1).unsqueeze(1)
    inputs = (inputs - im_means)/im_stds
    
    for i in range(len(inputs)):
        if math.isnan(torch.sum(inputs[i]).item()):
            print('found nan')
            
#     labels = labels.long().cuda()
#     optimizer.zero_grad()
#     outputs = model(inputs)
#     _, preds = torch.max(outputs.data, 1)

#     loss = criterion(outputs, labels)
#     if math.isnan(loss.item()):
#         break
        
#     loss.backward()
#     optimizer.step()
#     running_loss += loss.item()
#     num_iters += 1
#     if num_iters % 10 == 0:
#         print(loss.item())

  0%|          | 0/929 [00:00<?, ?it/s]

In [46]:
# train_file = '../../dataset_lists_openmind/train_list_image_train_v5_shapenet_non_zero.txt'
# test_file = '../../dataset_lists_openmind/test_list_image_train_v5_shapenet_non_zero.txt'

# train_files_cleaned = []
# test_files_cleaned = []

# with open(train_file,'r') as F:
#     content = F.readlines()
# train_files = [c.rstrip() for c in content]

# with open(test_file,'r') as F:
#     content = F.readlines()
# test_files = [c.rstrip() for c in content]

# train_files_cleaned = [i for i in train_files if i not in leftover_bad_files]
# test_files_cleaned = [i for i in test_files if i not in leftover_bad_files]

In [47]:
# len(train_files)

In [48]:
# len(train_files_cleaned)

In [49]:
# new_train_file = '../../dataset_lists_openmind/train_list_image_train_v5_shapenet_zero_removed.txt'
# new_test_file = '../../dataset_lists_openmind/test_list_image_train_v5_shapenet_zero_removed.txt'

In [50]:
# with open(new_train_file,'w') as F:
#     for i in train_files_cleaned:
#         print(i, file =F)

In [51]:
# with open(new_test_file,'w') as F:
#     for i in test_files_cleaned:
#         print(i, file =F)

In [52]:
# total_corrects = 0
# count = 0
# for data in tqdm(dset_loaders['test']):
#     inputs, labels, image_paths = data
#     inputs = inputs.float().cuda()
#     labels = labels.long().cuda()
#     outputs = model(inputs)
#     corrects = torch.sum(torch.argmax(outputs, dim=1) == labels).item()
#     total_corrects += corrects
#     count += len(inputs)

In [ ]:
print(model)
criterion = nn.CrossEntropyLoss()
optimizer_ft = optim.Adam(model.parameters(), lr = wandb_config['base_lr'])

if wandb_config['use_gpu']:
    criterion.cuda()
    model.cuda()

In [ ]:
###########################################################



best_final_model = model_pipeline(model, criterion, optimizer_ft, wandb_config)

save_path = '/om5/user/smadan/differentiable_graphics_ml/training_models/saved_models/%s_final.pt'%wandb_config['run_name']

with open(save_path,'wb') as F:
    torch.save(best_final_model,F)